In [49]:
import pandas as pd
import numpy as np

In [ ]:
dataset = 'amazon-google'
model = 'timesfm'
unit = 'h'
unit_num = 1
confidence = 0.80
quantiles = [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]

results_fn = f"model_results/{dataset}/{model}/median_preds.csv"
lower_fn = f"model_results/{dataset}/{model}/quantile_{round((1-confidence)/2 * 100)}_preds.csv"
upper_fn = f"model_results/{dataset}/{model}/quantile_{round((1+confidence)/2 * 100)}_preds.csv"
data_fn = f"data/{dataset}/y_{dataset}.csv"
results_df = pd.read_csv(results_fn, index_col=0, parse_dates=['ds'])
upper_df = pd.read_csv(upper_fn, index_col=0, parse_dates=['ds'])
lower_df = pd.read_csv(lower_fn, index_col=0, parse_dates=['ds'])
data_df = pd.read_csv(data_fn, index_col=0, parse_dates=['ds'])
freq_delta = pd.Timedelta(unit_num, unit=unit)
    
quantiles_df = []
for quantile in quantiles:
    quantile_fn = f"model_results/{dataset}/{model}/quantile_{round(quantile * 100)}_preds.csv"
    quantiles_df.append(pd.read_csv(quantile_fn, index_col=0, parse_dates=['ds']))
quantiles_df.insert(len(quantiles_df)//2, results_df)
quantiles.insert(len(quantiles)//2, 0.5)
quantiles_dict = dict(zip(quantiles, quantiles_df))

In [67]:
def mae(results_df, data_df, freq_delta):
    pred_length = int(results_df.columns[-1])
    mae_arr = []
    for h in range(1,pred_length+1):
        shift_results = results_df[['ds', 'unique_id', str(h)]]
        shift_results.loc[:,'ds'] += freq_delta * h
        merged_results = pd.merge(data_df, shift_results, on=['unique_id', 'ds'], how='inner')
        mean_abs_error = np.mean(np.abs(merged_results['y'] - merged_results[str(h)]))
        mae_arr.append(mean_abs_error)
    return np.mean(mae_arr), mae_arr

print(mae(results_df, data_df, freq_delta))

(32.12681547242537, [22.077080779814814, 25.660550988570986, 27.34293932852623, 28.16868018875, 28.610126772083337, 28.933602221844133, 29.96733481740741, 29.896822874852084, 30.246807435135036, 30.76492315385802, 30.95993794641204, 31.029039888819444, 31.206858219178784, 31.375631417785495, 31.37681332841821, 31.23893838040895, 31.44653217802778, 31.598738645219136, 31.44456859894136, 31.48625241433827, 31.76348530055787, 31.955432601554016, 31.963281362079474, 31.993610947638885, 32.749623208858026, 33.405168704587965, 33.61271430152778, 33.75478009709105, 33.49288005158179, 33.661591921083335, 34.45338756320988, 34.10192011959105, 34.027278929529324, 34.53487929197531, 34.28585305984568, 34.645457144228395, 34.324699758155866, 34.24288591692902, 34.379803464876545, 34.03927622417439, 34.342243651473765, 34.31854740491513, 34.464096213333335, 34.50992990921296, 34.50555705829476, 34.74427975371914, 34.55578180525463, 34.42651733274692])


In [ ]:
def mase(results_df, data_df, freq_delta):
    pred_length = int(results_df.columns[-1])
    mae_arr = []
    for h in range(1,pred_length+1):
        shift_results = results_df[['ds', 'unique_id', str(h)]]
        shift_results.loc[:,'ds'] += freq_delta * h
        merged_results = pd.merge(data_df, shift_results, on=['unique_id', 'ds'], how='inner')
        mean_abs_error = np.mean(np.abs(merged_results['y'] - merged_results[str(h)]))
        mae_arr.append(mean_abs_error)
    
    # naive mae
    shift_results = data_df.copy()
    shift_results.loc[:, 'ds'] -= freq_delta
    shift_results = shift_results.rename(columns={"y": "1"})
    merged_results = pd.merge(data_df, shift_results, on=['unique_id', 'ds'], how='inner')
    mae_n = np.mean(np.abs(merged_results['y'] - merged_results["1"]))
    return np.mean(mae_arr) / mae_n, np.array(mae_arr) / mae_n

print(mase(results_df, data_df, freq_delta))

In [ ]:
def tce(lower_df, upper_df, data_df, freq_delta, confidence):    
    pred_length = int(results_df.columns[-1])
    outside_ratio = (1-confidence)/2
    tce_arr = []
    for h in range(1,pred_length+1):
        shift_lower = lower_df[['ds', 'unique_id', str(h)]]
        shift_lower.loc[:,'ds'] += freq_delta * h
        shift_upper = upper_df[['ds', 'unique_id', str(h)]]
        shift_upper.loc[:,'ds'] += freq_delta * h
        merged_upper = pd.merge(data_df, shift_upper, on=['unique_id', 'ds'], how='inner')
        merged_lower = pd.merge(data_df, shift_lower, on=['unique_id', 'ds'], how='inner')
        mean_upper_outside = np.mean(merged_upper['y'] > merged_upper[str(h)])
        mean_lower_outside = np.mean(merged_lower['y'] < merged_lower[str(h)])
        tce_arr.append(abs(outside_ratio - mean_upper_outside) + abs(outside_ratio - mean_lower_outside))
    return np.mean(tce_arr), np.array(tce_arr)
print(tce(lower_df, upper_df, data_df, freq_delta, confidence))

0.09999999999999998
(0.02816679526748971, array([0.02785494, 0.02916667, 0.01712963, 0.01342593, 0.01466049,
       0.00902778, 0.01905864, 0.02623457, 0.03395062, 0.03009259,
       0.01820988, 0.02631173, 0.03024691, 0.01427469, 0.02901235,
       0.01535494, 0.01867284, 0.03742284, 0.01983025, 0.01682099,
       0.02669753, 0.01890432, 0.02523148, 0.0287037 , 0.02314815,
       0.03402778, 0.03016975, 0.02376543, 0.03263889, 0.0279321 ,
       0.04575617, 0.04475309, 0.0382716 , 0.04066358, 0.03279321,
       0.03626543, 0.04683642, 0.03996914, 0.0400463 , 0.03070988,
       0.0212963 , 0.04058642, 0.02530864, 0.03233025, 0.02947531,
       0.03155864, 0.02716049, 0.03024691]))


In [93]:
def wql(quantiles_dict, data_df, freq_delta):
    '''
    returns: weighted quantile loss, (n_quantiles, pred_length) WQL array
    '''
    ql_arr = []
    for quantile, quantile_df in quantiles_dict.items():
        quantile_ql_arr = []
        pred_length = int(quantile_df.columns[-1])
        for h in range(1,pred_length+1):
            shift_results = results_df[['ds', 'unique_id', str(h)]]
            shift_results.loc[:,'ds'] += freq_delta * h
            merged_results = pd.merge(data_df, shift_results, on=['unique_id', 'ds'], how='inner')
            quantile_loss = np.mean((2*(1-quantile)*(merged_results[str(h)] - merged_results['y'])*(merged_results[str(h)] >= merged_results['y'])) \
                            + (2*(quantile)*(merged_results['y'] - merged_results[str(h)])*(merged_results[str(h)] < merged_results['y'])))
            quantile_ql_arr.append(quantile_loss)
        ql_arr.append(quantile_ql_arr)

    scale = np.sum(merged_results['y'])
    wql_arr = np.array(ql_arr) / scale
    return np.sum(wql_arr), wql_arr
            
wql_val, wql_arr = wql(quantiles_dict, data_df, freq_delta)
print(wql_val, wql_arr.shape, np.mean(wql_arr))

0.0038484083119834514 (9, 48) 8.908352574035768e-06


In [ ]:
def msis(lower_df, upper_df, data_df, freq_delta, confidence):    
    pred_length = int(results_df.columns[-1])
    mis_arr = []
    for h in range(1,pred_length+1):
        shift_lower = lower_df[['ds', 'unique_id', str(h)]]
        shift_lower.loc[:,'ds'] += freq_delta * h
        shift_upper = upper_df[['ds', 'unique_id', str(h)]]
        shift_upper.loc[:,'ds'] += freq_delta * h
        merged_upper = pd.merge(data_df, shift_upper, on=['unique_id', 'ds'], how='inner')
        merged_lower = pd.merge(data_df, shift_lower, on=['unique_id', 'ds'], how='inner')
        mean_interval_score = np.mean( (merged_upper[str(h)] - merged_lower[str(h)]) \
                                      + confidence * (merged_lower[str(h)] - merged_lower['y']) * (merged_lower['y'] < merged_lower[str(h)]) \
                                      + confidence * (merged_upper['y'] - merged_upper[str(h)]) * (merged_upper['y'] > merged_upper[str(h)]) )
        mis_arr.append(mean_interval_score)
    
    # naive mae
    shift_results = data_df.copy()
    shift_results.loc[:, 'ds'] -= freq_delta
    shift_results = shift_results.rename(columns={"y": "1"})
    merged_results = pd.merge(data_df, shift_results, on=['unique_id', 'ds'], how='inner')
    mae_n = np.mean(np.abs(merged_results['y'] - merged_results["1"]))

    return np.mean(mis_arr) / mae_n, np.array(mis_arr) / mae_n

print(msis(lower_df, upper_df, data_df, freq_delta, confidence))

(1.9698064437374196, array([1.59909541, 1.69047787, 1.7798634 , 1.82855429, 1.83496945,
       1.86386496, 1.89763052, 1.89564952, 1.87658962, 1.88554238,
       1.88561317, 1.90186529, 1.92080159, 1.9247371 , 1.93812584,
       1.96945075, 1.97422951, 1.94015624, 1.94902403, 1.94419349,
       1.97811766, 1.96131106, 1.97714109, 1.96569066, 1.98812767,
       2.00991368, 2.01288925, 2.04183722, 2.06745672, 2.06035704,
       2.0416558 , 2.04050521, 2.03891777, 2.02324464, 2.05736086,
       2.0875128 , 2.09095392, 2.0825263 , 2.08135843, 2.08403165,
       2.04955599, 2.0415112 , 2.03088281, 2.0284644 , 2.04071957,
       2.04759876, 2.05401842, 2.06661426]))
